# Herramientas para proyecto de lenguaje natural

# Herramienta para extraer PDFS a partir de palabras clave (Se hace un scrap general en google), los PDFS serán utilizados para alimentar el modelo

In [9]:
#Herramienta para scrapear pdfs con palabras clave 
import os
import requests
from bs4 import BeautifulSoup
from googlesearch import search

def search_pdfs(keywords, num_results=10): #función para extraer pdfs en el directorio indicado
    query = f"{keywords} filetype:pdf"
    urls = []
    for url in search(query, num_results=num_results):
        if url.endswith(".pdf"):
            urls.append(url)
    return urls

def download_pdf(url, output_dir):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        filename = os.path.join(output_dir, url.split("/")[-1])
        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"Descargado: {filename}")
    except requests.RequestException as e:
        print(f"Error al descargar {url}: {e}")

if __name__ == "__main__":
    output_dir = "C:/Users/angel/OneDrive/Escritorio/IA/LenguajeNatural/pdfsExtracted" #directorio de salida
    os.makedirs(output_dir, exist_ok=True)
    print("Buscando PDFs...")
    pdf_urls = search_pdfs("Reforma poder judicial debate", num_results=10) #palabras clave y cantidad de resultados scrapeads
    
    if pdf_urls:
        print(f"Se encontraron {len(pdf_urls)} PDFs. Descargando...")
        for pdf_url in pdf_urls:
            download_pdf(pdf_url, output_dir)
    else:
        print("No se encontraron PDFs.")


# Herramienta para scrapear links de articulos con palabras clave (sitios para posteriormente extraer el contenido en un txt)

In [1]:
#Herramienta para encontrar articulos con palabras clave 
from googlesearch import search

def buscar_articulos(keywords, num_resultados=10):
    try:
        resultados = search(keywords, num_results=num_resultados, lang="es")
        return list(resultados) #imprime los links
    except Exception as e:
        print(f"Error al realizar la búsqueda: {e}")
        return []

def guardar_en_txt(articulos, archivo="C:/Users/angel/OneDrive/Escritorio/IA/LenguajeNatural/resultados.txt"): #extrae los links en un txt (opcional esta función)
    try:
        with open(archivo, "w", encoding="utf-8") as f:
            f.write("Resultados de la búsqueda:\n\n")
            for idx, url in enumerate(articulos, start=1):
                f.write(f"{idx}. {url}\n")
        print(f"Resultados guardados en el archivo: {archivo}")
    except Exception as e:
        print(f"Error al guardar los resultados en el archivo: {e}")

if __name__ == "__main__":
    articulos = buscar_articulos("Reforma al poder judicial noticias", 10) #palabras clave y cantidad de elementos a scrapear de google
    if articulos:
        print("\nArtículos encontrados:")
        for idx, url in enumerate(articulos, start=1):
            print(f"{idx}. {url}")
        guardar_en_txt(articulos)
    else:
        print("No se encontraron resultados.")



Artículos encontrados:
1. https://elpais.com/noticias/reforma-judicial/
2. https://ibero.mx/prensa/mexico-atraviesa-una-crisis-constitucional-con-la-reforma-al-poder-judicial-especialistas
3. https://www.tallapolitica.com.mx/dan-primera-lectura-a-las-leyes-secundarias-de-la-reforma-al-poder-judicial/
4. https://justiciatv.mx/noticias/termino/189
5. https://www.nmas.com.mx/temas/poder-judicial/
6. https://www.reforma.com/avanzan-leyes-secundarias-de-reforma-judicial-en-senado/ar2917910
7. https://www.gob.mx/presidencia/prensa/reforma-al-poder-judicial-fortalece-el-estado-de-derecho-inversiones-en-mexico-estan-seguras-presidenta-claudia-sheinbaum?idiom=es
8. https://elpais.com/mexico/2024-11-05/mexico-elegira-a-sus-jueces-por-voto-popular.html
9. https://www.milenio.com/temas/reforma-judicial
10. https://www.bbc.com/mundo/articles/cyvpdneyzlgo
Resultados guardados en el archivo: C:/Users/angel/OneDrive/Escritorio/IA/LenguajeNatural/resultados.txt


# Herramienta para extraer el texto principal de un sitio web (sitios de noticias, blogs, etc), genera un txt para alimentar al modelo

In [2]:
#Herramienta para extraer el texto de páginas web (por ejemplo, de noticias)
import requests
from bs4 import BeautifulSoup

def extraer_texto_relevante(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Buscar el contenedor principal del artículo (ajustar según el sitio web)
        articulo = soup.find('article')  
        if not articulo:
            articulo = soup.find('div', class_='main-content')  
        if not articulo:
            articulo = soup  # Si no se encuentra, usar todo el HTML
        
        # Eliminar barras laterales, anuncios y contenido no relevante
        for sidebar in articulo.find_all(['aside', 'nav', 'footer']):
            sidebar.decompose()  # Elimina estos elementos del DOM
        
        for no_relevante in articulo.find_all('div', class_=['sidebar', 'related', 'ads', 'promo', 'banner']):
            no_relevante.decompose()
        
        # Extraer encabezados y párrafos del artículo principal
        contenido = []
        for encabezado in articulo.find_all(['h1', 'h2', 'h3']):
            contenido.append(encabezado.get_text(strip=True))
        for parrafo in articulo.find_all('p'):
            contenido.append(parrafo.get_text(strip=True))
        
        texto_relevante = "\n".join(contenido)
        return texto_relevante

    except requests.RequestException as e:
        print(f"Error al acceder a la URL: {e}")
        return None

if __name__ == "__main__":
    url = "https://ibero.mx/prensa/mexico-atraviesa-una-crisis-constitucional-con-la-reforma-al-poder-judicial-especialistas" #se indica el sitio
    
    print("Extrayendo contenido...")
    texto = extraer_texto_relevante(url)
    
    if texto:
        print("\nTexto extraído:")
        print(texto)
        
        archivo = "C:/Users/angel/OneDrive/Escritorio/IA/LenguajeNatural/txtArticulosJudicial/articulo_extraidoJudicial_00.txt" #ruta y nombre de extracciónen txt
        with open(archivo, "w", encoding="utf-8") as f:
            f.write(texto)
        print(f"\nEl texto se ha guardado en: {archivo}")
    else:
        print("No se pudo extraer el contenido.")


Extrayendo contenido...

Texto extraído:
México atraviesa una crisis constitucional con la Reforma al Poder Judicial: especialistas

¿Por qué revisar la inconstitucionalidad de la Reforma al Poder Judicial?
DESTACADOS
SÍGUENOS:
UNIVERSIDAD IBEROAMERICANA CDMX
No podemos vendarnos los ojos, para ver que México atraviesa una crisis constitucionalporque ya no se sabe si una sentencia avalada por los ministros de la Suprema Corte de Justicia de la Nación (SCJN) vaya a ser acatada o no, porque tenemos una renuncia masiva dejueces y magistrados, y hay pronunciamientos desde la tribuna legislativa de los líderes parlamentarios que invitan a desacatar las resoluciones del Poder Judicial, afirmóDaniela Carrasco Berge, Secretaria de Estudio y Cuenta de Ponencia del Ministro Juan Luis González Alcántara Carrancá.
"El capitán (Poder Ejecutivo) no es el dueño del barco, que se elige por la mayoría de los tripulantes (ciudadanos). Están los encargados de mantener ese barco (Poder Legislativo) y está

# Herramienta para calcular similitud coseno (Observar que tanto se parecen los textos entre sí a partir de una ruta, con la finalidad de diversificar el contenido al respecto y tener mucha más informaión, y también que se encuentren en un mismo contexto)

In [3]:
#Herramienta para saber que tan parecidos son entre sí los textos (tener mayor diversidad de info)

import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2
import glob
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words_espanol = stopwords.words('spanish')  #

def leer_pdf(ruta):
    texto = ""
    try:
        with open(ruta, 'rb') as archivo:
            lector = PyPDF2.PdfReader(archivo)
            for pagina in lector.pages:
                texto += pagina.extract_text()
    except Exception as e:
        print(f"Error al leer {ruta}: {e}")
    return texto

def leer_txt(ruta):
    texto = ""
    try:
        with open(ruta, 'r', encoding='utf-8') as archivo:
            texto = archivo.read()
    except Exception as e:
        print(f"Error al leer {ruta}: {e}")
    return texto

def recopilar_documentos(directorio):
    documentos = []
    archivos = glob.glob(os.path.join(directorio, "*.pdf")) + glob.glob(os.path.join(directorio, "*.txt"))
    
    for archivo in archivos:
        if archivo.endswith(".pdf"):
            documentos.append(leer_pdf(archivo))
        elif archivo.endswith(".txt"):
            documentos.append(leer_txt(archivo))
    return documentos, archivos

# Calcular la similitud coseno
def calcular_similitud(documentos):
    # Convertir los documentos en vectores TF-IDF usando las stop words en español
    vectorizador = TfidfVectorizer(stop_words=stop_words_espanol)
    tfidf_matriz = vectorizador.fit_transform(documentos)
    
    # Calcular la similitud coseno
    similitud = cosine_similarity(tfidf_matriz)
    return similitud

def main():
    directorio = "C:/Users/angel/OneDrive/Escritorio/IA/LenguajeNatural/judicial"  
    documentos, nombres_archivos = recopilar_documentos(directorio)

    if not documentos:
        print("No se encontraron documentos.")
        return

    similitud = calcular_similitud(documentos)

    print("Matriz de similitud coseno:")
    for i, nombre1 in enumerate(nombres_archivos):
        for j, nombre2 in enumerate(nombres_archivos):
            if i != j:
                print(f"Similitud entre '{os.path.basename(nombre1)}' y '{os.path.basename(nombre2)}': {similitud[i][j]:.2f}")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Matriz de similitud coseno:
Similitud entre '128260.pdf' y '240910_PPT_PJ__CS.pdf': 0.17
Similitud entre '128260.pdf' y 'analisis comparativo.pdf': 0.41
Similitud entre '128260.pdf' y 'Análisis de la iniciativa de reforma. Problemas asociados_final (1).pdf': 0.43
Similitud entre '128260.pdf' y 'Análisis de la iniciativa de reforma. Problemas asociados_final.pdf': 0.43
Similitud entre '128260.pdf' y 'asun_4780554_20241008_1728416248.pdf': 0.23
Similitud entre '128260.pdf' y 'Comites-de-evaluacion-PJ-.docx.pdf': 0.23
Similitud entre '128260.pdf' y 'Consulta%20a%20tr%C3%A1mite%204-2024.pdf': 0.37
Similitud entre '128260.pdf' y 'debate_poder_judicial.pdf': 0.24
Similitud entre '128260.pdf' y 'Estudio-sobre-la-Reforma-Judicial-version-completa.pdf': 0.43
Similitud entre '128260.pdf' y 'La-reforma-judicial-de-AMLO.pdf': 0.20
Similitud entre '128260.pdf' y 'Ley al poder judicial news.pdf': 0.18
Similitud entre '128260.pdf' y 'Ley-Organica-PJF-20231027.pdf': 0.58
Similitud entre '128260.pdf'

# Preguntas Ley al poder Judicial

# ¿El Diagnostico de la ley al poder judicial es conocido y qué estudios expertos tuvieron en cuenta?

In [ ]:
Respuesta dada por el modelo:

El Diagnóstico de la Ley al Poder Judicial, publicado por el Consejo Nacional de Justicia y Derechos Humanos, es un documento que revisa
las condiciones internas del Poder Judicial para evaluar su eficacia y mejorar su funcionamiento.

Los estudios expertos incluyen a diversos especialistas en derecho judicial, sociología, política y otras áreas.
Algunas de estas recomendaciones fueron:

*   Rendición de Cuentas: Se sugiere que se implementen mecanismos más efectivos para garantizar la rendición de cuentas.
*   Transparencia y Rendición de Cuentas: Se recomienda mejorar la transparencia en los procesos de toma de decisiones 
    y proporcionar información clara a los ciudadanos sobre el funcionamiento del Poder Judicial.
*   Responsabilidad y Rendición de Cuentas: Se sugiere establecer mecanismos de responsabilidad claros entre empleados, 
    ciudadanos, empleados del Estado y demás actores interesados.

El Diagnóstico destaca que la ley al poder judicial debe ser más flexible y adaptable para responderte 
a los cambios en las circunstancias políticas, sociales e institucionales.

Fundamento: Estudio-sobre-la-Reforma-Judicial-version-completa, Análisis de la iniciativa de reforma. Problemas asociados_final 
    

# ¿Por qué la reforma no incluyó a las fiscalías a las defensoría y sólo se limitó al poder judicial ?

In [ ]:
Respuesta dada por el modelo:
La reforma tampoco incluyó a las fiscalías de defensa en el sentido tradicional, ya que:

*   Las fiscalías son organismos independientes del Poder Judicial;
*   El objetivo principal de la reforma era mejorar el funcionamiento del Poder Judicial, no reducir su autonomía.

Sin embargo, considerando desde una perspectiva más amplia y las características actuales y los objetivos pretendidos:

1.  **La estructura de poder**: Se mantuvo la estructura tradicional dentro del sistema jurisdiccional para 
    garantizar su capacidad para tomar decisiones.
2.  **Objetivos políticos**: La reforma se centró en mejorar el funcionamiento del Poder Judicial, 
    pero se limitaron a incluir a las fiscalías de defensa debido a su autonomía política y profesional.

Fundamentos: Reforma del Poder judicial mapa urgente para el debate, articulo_extraidoJudicial_24.txt

# ¿Qué medidas concretas se van a implementar para evitar la captación del crimen organizado y la violencia en el contexto electoral?

In [ ]:
Respuesta dada por el modelo:
Para prevenir la captación de los criminales organizados y la violencia en el proceso electoral, las siguientes medidas se van a implementar:

*   **Desarrollo de capacidades**: Se proporcionará capacitación especializada a las fuerzas de seguridad para 
    mejorar su capacidad de contener los disturbios y la violencia.
*   **Seguridad física**: Se instalarán cámaras de seguridad con reconocimiento por el Big Brother en los centros de votación,
    así como instalaciones de control de acceso, para prevenir el acceso ilegal al lugar de votación.

Fundamentos: Análisis de la iniciativa de reforma. Problemas asociados_final (1), articulo_extraidoJudicial_16.txt

# ¿Cómo garantizar que juristas probos y honestos se animen a competir públicamente frente a los riesgos de la violencia y que la campaña para ser incluidos en las listas no implique negociaciones indebidas?

In [ ]:
Respuesta dada por el modelo:
En este sentido, se pueden implementar varios pasos para garantizar que los magistrados y jueces sean elegidos sin manipulaciones indebidas:

*   **Sistema de selección transparente**: Se establecerá un sistema de selección transparente donde la competencia se 
    basa en su meritocracia, honestidad, integridad e incluidas las habilidades necesarias para desempeñar el cargo.

*   **Fomento de la independencia**: Los magistrados y jueces serán proactivamente fomentado un ambiente de respeto
    por la independencia y la autonomía profesional para asegurar que se mantengan en su rol público sin influencias indebidas.
        
*   **Investigación independiente**: Se realizará una investigación independiente sobre cualquier posible irregularidad
    o manipulación durante el proceso electoral, e investigaciones adicionales serán realizadas a medida de lo necesario.
        
*   **Evaluación del rendimiento de los magistrados y jueces**: Estos se evaluarán periódicamente para asegurarse que 
    siguen cumpliendo su función pública sin interferencias indebidas y así evitar cualquier manipulación o influencia sobre su comportamiento.

        
Fundamentos: poderJudicial.txt, articulo_extraidoJudicial_7.txt, articulo_extraidoJudicial_1.txt

# ¿Cómo se conforman estos comités de postulación?

In [ ]:
Respuesta dada por el modelo:

Estos comités de postulación estarán formados por representantes de diversas instituciones del Estado y la sociedad 
civil que participen en el proceso electoral. Ellos contribuirán a crear un comité de postulación eficiente e integral 
que albergue la competencia entre los candidatos y evalúe sus propuestas en función de la ley establecida para su aplicación.

El comité de postulación estará compuesto por:

*   **Representantes institucionales:** Los representantes de las instituciones electorales, jurídicas,
    académicas y otras que contribuyen al proceso electoral.

*   **Representantes profesionales:** Representarán los grupos profesionales como abogados, jueces y otros.
        
*   **Representantes sociales civiles:** Estos participantes incluirían a las organizaciones de la sociedad civil 
    que ejercen un gran impacto en la sociedad.
        
Fundamentos: poderJudicial.txt, articulo_extraidoJudicial_7.txt, articulo_extraidoJudicial_1.txt

# ¿Cómo asegurar la carrera judicial?

In [ ]:
Respuesta dada por el modelo:

Se pretende mantener la carrera judicial sostenible mediante diversas medidas, como:

*   **Establecer plazos de tiempo máximos:** Se fijarán los plazos para que los jueces resuelvan juicios; ello contribuirá a 
    preservar el tiempo disponible para tomar decisiones y garantizar la eficiencia.

*   **Evaluación de la carrera judicial**: se llevarán a cabo auditorías periódicas para evaluar la calidad y la eficacia del sistema judicial. 

*   **Reducción del número de jueces en proceso:** Se considerarían posibles reducciones del número de jueces que permanecen en el proceso.

*   **Ajustes en la estructura del poder judicial**: se podrían ajustar las proporciones entre diferentes tipos de jueces, 
    como los jueces ordinarios y los jueces supremos.
*   **Mejoras en la relación con otros instituciones judiciales:** Se facilitaría la colaboración entre el Poder Judicial y otras 
    instituciones del Estado para mejorar las relaciones e interacciones dentro del sistema.
        
Fundamentos: LEY-ORGÁNICA-DEL-PODER-JUDICIAL-REF-5-DE-JULIO-DE-2023, Estudio-sobre-la-Reforma-Judicial-version-completa, 128260.pdf, 
             Reforma del Poder judicial mapa urgente para el debate

# ¿Cómo compatibilizar la incorporación de medidas para preservar la identidad de los jueces conocidos en el sistema interamericano como "jueces sin rostro" con los estándares interamericanos ?


In [ ]:
Respuesta dada por el modelo:

Para establecer una posición racional sobre este tema, considerando las preocupaciones éticas y legales involucradas.

Al incorporar medidas para preservar la identidad de jueces en el sistema interamericano, es crucial que se adopten pasos concretos y 
estructurados para garantizar la transparencia, la justicia y los derechos de las partes. Esto incluiría:

1.  **Aclarar la definición de "jueces sin rostro":** Se debe aclarar qué características y acciones consideran "jueces sin rostro" y
cómo se aplicará esta definición en diferentes contextos.

2.  **Establecer un marco legal claro:** Los organismos internacionales deben establecer claramente las leyes y procedimientos que rigen 
el acceso al sistema judicial y la identidad de los jueces, con el fin de evitar abusos y garantizar la equidad.

3.  **Garantizar la transparencia y la participación ciudadana:** Se debe garantizar que la información necesaria para determinar
la identidad de los jueces esté disponible en forma accesible a todas las partes interesadas, y se deben implementar procedimientos
transparentes y justos para que las partes puedan acceder a información relevante sobre su caso.

4.  **Proporcionar protección adecuada:** Se debe proporcionar una protección adecuada para los derechos de las partes, incluyendo 
la posibilidad de obtener asesoramiento legal adicional si se sospecha un error en el acceso al sistema judicial o en la identidad del juez.

5.  **Fomentar la participación de las partes interesadas:** Se debe promover la participación activa de las partes interesadas, 
incluyendo a los litigantes, a la comunidad y a los grupos representativos de personas vulnerables, para garantizar que sus intereses 
sean considerados en el proceso judicial.

En última instancia, es crucial que estas medidas sean adoptadas de manera que se respeten los principios interamericanos de derechos 
humanos y las normas legales vigentes.
        
Fundamentos: extraido.txt,articulo_extraidoJudicial_6.txt

# ¿Cómo impactará el enorme costo económico que tendrá la implementación de esta reforma con la promoción y ?

In [ ]:
Respuesta dada por el modelo:

La implementación de la reforma podría tener un impacto significativo en los gobiernos y las organizaciones involucradas, 
    incluyendo un aumento de los costos relacionados con el financiamiento para promover la reforma. 
    Sin embargo, también se podría apreciar un beneficio importante: reducir las pérdidas económicas en los sectores afectados
    por la implementación de una nueva política judicial.
        
Fundamentos: articulo_extraidoJudicial_14.txt, articulo_extraidoJudicial_8.txt,articulo_extraidoLeyOrganismosAutonomos_6.txt

# Ley Organismos Autónomos

# ¿Es constitucional esta ley, considerando que algunos organismos autónomos están establecidos en la Constitución?

In [ ]:
Respuesta dada por el modelo:

La Constitución mexicana permite la creación y regulación de instituciones autónomas para abordar problemas específicos, 
siempre que cumplan con los requisitos legales y éticos establecidos en el texto de la Constitución.

Sin embargo, la implementación de una ley que elimina completamente las instituciones autónomas podría verse como un acto 
autoritario o totalitario del Estado.

En este sentido, si se considera que algunas instituciones autónomas están establecidas en la Constitución, 
la eliminación de esas instituciones también podría verse como un acto constitucionalmente inaceptable. La Constitución mexicana es un texto fundamental y riguroso que busca garantizar los derechos y libertades fundamentales de todos los ciudadanos.

La Constitución establece que:

*   El Estado se reserva el derecho de controlar y supervisar a las instituciones autónomas.
*   Los órganos autónomos deben tener una autonomía institucional propia, reconocida por el Estado.

En resumen, si la ley eliminaría completamente las instituciones autónomas, podría verse como un acto 
autoritario que viola los principios y condiciones establecidos en la Constitución.
        
Fundamentos: organismos autonomos en méxico.pdf


# ¿Cómo afectaría la eliminación de estos organismos a la transparencia y rendición de cuentas en el gobierno?

In [ ]:
Respuesta dada por el modelo:

La eliminación de instituciones autónomas puede tener un impacto significativo en la transparencia y rendición de cuentas en el gobierno. 
Al eliminar los mecanismos de control y supervisión que permiten a las instituciones autónomas informar sobre su funcionamiento, 
también se reduce la capacidad de los ciudadanos para saber cómo funcionan los gobiernos públicos.

En este sentido, la eliminación de estas instituciones autónomas podría verse como un acto que:

*   Reduce la independencia y autonomía de los órganos judiciales y legales.
*   Aumenta la dependencia del Estado para controlar a estos organismos.
*   Disminuye la capacidad de las instituciones autónomas para informar sobre su funcionamiento y tomar decisiones.
*   Reduce la transparencia en el proceso gubernamental.

La eliminación de estas instituciones autónomas también podría tener un impacto significativo en la rendición de
    cuentas del gobierno. Al reducir la capacidad de los órganos judiciales para informar sobre su funcionamiento,
    se reduce la capacidad de estos órganos para realizar auditorías y informes sobre el uso de fondos públicos.

En resumen, la eliminación de instituciones autónomas puede tener un impacto significativo en la transparencia y rendición de cuentas en el gobierno.
        
Fundamentos: organismos autonomos en méxico.pdf, articulo_extraidoLeyOrganismosAutonomos_5.txt


# ¿Qué funciones críticas podrían perder independencia y control al pasar al poder ejecutivo o a otras instituciones?

In [ ]:
Respuesta dada por el modelo:

Las funciones críticas que son inherentemente independientes y sujetas a un control limitado pueden enfrentar una reducción
en su autonomía al pasar al poder ejecutivo o a otras instituciones. Esto puede llevar a diferentes escenarios, dependiendo de
la naturaleza del funcionamiento de la institución y el modelo político que se está implementando.

*   **Control ejecutivo**: En este caso, las funciones críticas pueden perder su autonomía si son integradas a un órgano ejecutivo.
Esto puede implicar una mayor interconexión entre diferentes niveles del gobierno o una mayor dependencia de la opinión pública para tomar decisiones.

*   **Dependencia política**: En este caso, las funciones críticas pueden perder su autonomía si son controladas por el poder 
    político de un grupo específico. Esto puede implicar una mayor influencia política en la toma de decisiones o una menor capacidad
    para adoptar políticas públicas.
        
*   **Dependencia externa**: En este caso, las funciones críticas pueden perder su autonomía si son sujetas a revisión y control 
    externo. Esto puede implicar una mayor influencia de organizaciones internacionales o extranjeras en la toma de decisiones.
        
*   **Reducción del acceso a información clasificada**: En este caso, las funciones críticas pueden perder su autonomía si no 
    tienen acceso a información confidencial y clasificada. Esto puede implicar una mayor dependencia del gobierno para obtener esta información.

En resumen, la reducción en la autonomía de las funciones críticas al pasar a un órgano ejecutivo o a otras instituciones puede tener
        diferentes efectos, dependiendo de la naturaleza del funcionamiento y el modelo político que se está implementando.

Fundamentos:poderJudicial.txt,articulo_extraidoJudicial_7.txt, articulo_extraidoJudicial_1.txt


# ¿Existen alternativas para mejorar la eficiencia de los organismos autónomos sin eliminarlos?

In [ ]:
Respuesta dada por el modelo:

La mejora de la eficiencia en órganos autónomos puede lograrse mediante diversas estrategias que no involucren
la eliminación de estos órganos. Aquí se presentan algunas alternativas:

1.  **Construcción de mecanismos de control y fiscalización**: Estas herramientas pueden ayudar a garantizar que las funciones 
    críticas estén cumpliendo con sus obligaciones y no se vean afectadas por decisiones políticas o económicas.

2.  **Creación de mecanismos de transparencia y rendición de cuentas**: Estos mecanismos pueden permitir a la sociedad
    conocer más sobre las actividades de los órganos autónomos y su relación con el gobierno, lo que puede contribuir a una mayor confianza en ellos.
        
3.  **Desarrollo de sistemas de gestión descentralizada**: Estos sistemas pueden ayudar a reducir la dependencia de los órganos
    autónomos en la toma de decisiones y aumentar su autonomía en la administración de las funciones críticas.

4.  **Implementación de mecanismos de resolución de conflictos**: Estos mecanismos pueden ayudar a resolver disputas
    entre diferentes niveles del gobierno o organizaciones internacionales, lo que puede reducir la carga de trabajo de los
    órganos autónomos y mejorar su autonomía.

5.  **Diversificación de las funciones críticas**: En algunas casos, es posible ofrecer nuevas funciones que no se 
    puedan cumplir por las mismas personas, lo que puede reducir el carácter autónomo de la institución y mejorar su eficiencia.
        
6.  **Estudio de caso: Implementación de políticas públicas innovadoras**: Analizar cómo funcionaron las políticas públicas en
    países o regiones con sistemas similares puede proporcionar modelos para implementar en México, lo que puede ayudar a mejorar la 
    eficiencia en los órganos autónomos.
        
7.  **Reformas estructurales**: En algunos casos, es posible reformar las instituciones internas de una
    organización autónoma para mejorar su autonomía y reducir el impacto sobre las funciones críticas que se 
    han transferido a otros niveles del gobierno o instituciones externas.

Es importante tener en cuenta que cada caso es único y que la respuesta adecuada dependerá de los objetivos específicos 
de cada organización autónoma y del contexto político en el que operan.

Fundamentos: organismos autonomos en méxico.pdf


# ¿Qué sectores de la sociedad civil y grupos de interés se verían afectados por la desaparición de estos organismos?

In [ ]:
Respuesta dada por el modelo:

La desaparición de los órganos autónomos podría tener varios sectores de la sociedad civil y grupos de interés afectados, incluyendo:

1.  **Activistas de derechos humanos**: Son conocidos por su compromiso con la protección de los derechos fundamentales y 
    pueden sentirse frustrados o desaparecer si las funciones críticas de los órganos autónomos se vayan a centralizar.

2.  **Participantes en movimientos sociales**: Muchos de ellos pueden estar activamente involucrados en el proceso de formulación y
    implementación de políticas públicas que beneficiarían a la sociedad, lo que podría ser desafiantizado por la centralización de funciones.
        
3.  **Organizaciones no gubernamentales (ONG)**: Algunas ONG pueden estar integradas en los órganos autónomos o tienen objetivos directamente
    relacionados con las mismas, lo que podría afectar su capacidad para realizar sus actividades.

4.  **Instituciones académicas y de investigación**: Están capacitadas para analizar la situación política y social y pueden estar afectadas
    por la centralización de funciones si se desconectan de los órganos autónomos a través de las carreras o proyectos que realizaban.

5.  **Emprendedores e innovadores**: Pueden aprovechar la oportunidad de nuevas regulaciones para desarrollar nuevos modelos de negocio y 
    crear empleo en áreas como la tecnología, la logística o el servicio público.
        
6.  **Instituciones financieras y bancos**: Algunos pueden ser afectados por cambios en su papel en la economía, ya que los órganos 
    autónomos a menudo son instituciones financieras clave para ciertos sectores.

Estos grupos de interés pueden sentirse afectados de varias maneras:

*   Desaparición del poder y autonomía
*   Percepción de que están siendo excluidos del proceso político
*   Sentimiento de que sus necesidades y objetivos no se están considerando en el proceso de reforma
*   Pérdida de la confianza en las instituciones y sus procesos

Fundamentos: articulo_extraidoLeyOrganismosAutonomos_6.txt, organismos autonomos en méxico.pdf, articulo_extraidoLeyOrganismosAutonomos_1.txt


# Opinión a base de la investigación realizada

In [ ]:
Mi opinión respecto al controversial tema respecto a la reforma al poder judicial y la eliminación de organismos autónomos es:

**En contra. Sí bien, anteriormente no tenía mucho contexto sobre el tema, ahora gracias a la investigación y extracción de información para alimentar 
al modelo, encontré que realmente tiene detalles que no me parecen muy correctos, algunas respuestas del modelo expresan esos rasgos como, la perdida 
de autonomía y voz por parte de instituciones externas al gobierno, que de cierta forma, fungían como intermediarios de la sociedad en general, 
un claro ejemplo es la desaparición del INAI, el cual era un medio de transparencia para el acceso público, lo cual ayudará a que se pueda dar paso
a usos indebidos de la información personal de la población al no haber un intermediario que supervice o se encarge de tales datos.

"Articulo judicial 31"- México aprueba la eliminación de siete organismos autónomos, que funcionaban como contrapesos al poder ejecutivo
México ha aprobado la reforma constitucional que permite la eliminación de siete órganos autónomos.
Los organismos que se eliminan incluyen el Instituto Nacional de Transparencia, Acceso a la Información y Protección de Datos Personales (INAI),
así como entidades encargadas de medir la pobreza, analizar los niveles educativos y regular sectores clave como el de la energía,
los hidrocarburos, las telecomunicaciones y la libre competencia.

"Articulo judicial 30"- Los organismos autónomos que se extinguirán son:
                        Instituto Nacional de Transparencia, Acceso a la Información y Protección de Datos Personales
                        Consejo Nacional de Evaluación de la Política de Desarrollo Social
                        Comisión Federal de Competencia Económica
                        Instituto Federal de Telecomunicaciones
                        Comisión Nacional de Mejora Continua de la Educación
                        Comisión Reguladora de Energía
                        Comisión Nacional de Hidrocarburos.
                        Las atribuciones de estos organismos pasarán a dependencias del Gobierno federal.


**Así mismo, el hecho de hacer a elección pública cargos tan relevantes y a la vez tan peligrosos en temas de privacidad de las personas postuladas, 
me resulta no muy óptimo, dado que será un tiempo de elecciones muy pesado, dudo que sea práctico y nuevamente, conllevará riesgos para los postulados

"Articulo judicial 1"- México elegirá a partir del próximo año a todos sus jueces por voto popular. 
                       Se trata de miles de cargos judiciales de nivel federal y estatal. Alrededor de 1.700 cargos 
                       judiciales de nivel federal tendrán que ser votados en las urnas en dos elecciones, una en 2025 y otra en 2027.

    



Finalmente, no estoy de acuerdo dado que el cambio de las reformas y la eliminación de estos organismos me genera la impresión de el deseo
del nuevo gobieno de querer centralizar el poder en un único lugar (morena en general), además de que se encuentra en gran parte de los estados, 
comienzan
a dejar al país sin herramientas para su autonomía y poder ejercer un poder "único" proveniente de este gobierno.


"Articulo judicial 28" - La más reciente propuesta de reforma del presidente Andrés Manuel López Obrador 
                        está en el debate público en México.Centrada en los Organismos Constitucionales Autónomos(OCAs),
                        esta iniciativa busca una transformación radical que podría redefinir el panorama de la regulación y 
                        supervisión gubernamental en diversas áreas críticas de la política y economía mexicanas.

"Articulo judicial 25"- El expresidente de México (1994-2000), Ernesto Zedillo, consideró este domingo que la reforma al Poder Judicial, 
                        busca la destrucción de su independencia para ponerlo al servicio de la fuerza política en el poder.


# Opinión del modelo

In [ ]:
Se le preguntó al modelo sobre su postura respecto al tema, y esta fue su respuesta:

La reforma judicial en México se está llevando a cabo para abordar desafíos como la ineficiencia de algunos tribunales,
la falta de transparencia y la necesidad de mejorar la justicia. La idea de eliminar organismos autónomos que consideran que 
son ineficientes o no funcionan bien puede ser una medida para revitalizar el proceso judicial.

Sin embargo, también es importante considerar las posibles consecuencias negativas de tal reforma, como:

*   **Impacto en la justicia social:** La reducción del papel de organismos autónomos podría llevar a una menor atención y
      resolución de casos que requieren una mayor participación ciudadana.
*   **Pérdida de expertise y conocimientos:** La eliminación de instituciones especializadas puede resultar en el 
      desvío de recursos y la pérdida de las habilidades y conocimientos necesarios para abordar los problemas sociales.

En resumen, mientras que la reforma judicial puede ser una medida para mejorar la justicia, también es importante
considerar las posibles consecuencias a largo plazo de tal cambio. Es crucial evaluar cuidadosamente las implicaciones 
éticas y sociales de cualquier medida para garantizar que se apoye un sistema justo, transparente e inclusivo.


Fundamentos: articulo_extraidoLeyOrganismosAutonomos_9.txt, Reforma del Poder judicial mapa urgente para el debate
